In [41]:
#!/usr/bin/env python

import concurrent.futures
import pandas as pd
import numpy as np
import urllib
import pathlib
import hashlib
import os
import sys
import cv2
import json
import tensorflow as tf
import PIL
import matplotlib.pyplot as plt
import keras
import scipy

from keras.preprocessing.image import ImageDataGenerator

import foolbox

IMG_HEIGHT = 224
IMG_WIDTH = 224

#image_generator = ImageDataGenerator(rescale=1./255)
#train_data = image_generator.flow_from_directory(batch_size=32,
#	directory=sys.argv[1],
#	target_size=(IMG_HEIGHT, IMG_WIDTH),
#	class_mode=None)

def load_image(filename):
	img = tf.keras.preprocessing.image.load_img(filename, target_size=(IMG_WIDTH,IMG_HEIGHT))
	img = tf.keras.preprocessing.image.img_to_array(img)
	img = np.expand_dims(img, axis=0) / 255
	return img

labels_file = open("labels.json", "r")
labels = json.loads(labels_file.read())

inverted_labels = {v: k for k, v in labels.items()}

model = tf.keras.models.load_model("model.h5")

#tf.keras.backend.set_learning_phase(0)
#preprocessing = dict(flip_axis=-1)
#fmodel = foolbox.models.KerasModel(model=model, bounds=(0.0,1.0), preprocessing=preprocessing)
fmodel = foolbox.models.KerasModel(model=model, bounds=(0.0,1.0), predicts="logits")

#keras.backend.set_learning_phase(0)
#kmodel = keras.applications.resnet50.ResNet50(weights='imagenet')
#preprocessing = dict(flip_axis=-1, mean=np.array([104, 116, 123]))  # RGB to BGR and mean subtraction
#fmodel = foolbox.models.KerasModel(kmodel, bounds=(0, 255), preprocessing=preprocessing)

model.summary()

img = load_image("persons-cropped/Donald Trump/122.jpg")



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0   

In [42]:
train_classes =os.listdir("persons-cropped")
test_classes = os.listdir("persons-cropped-test")

attack = foolbox.attacks.SaliencyMapAttack(fmodel)

def create_adv(img, label):
    #adv = scipy.ndimage.gaussian_filter(img, sigma=5)
    adv = attack(img, np.array([label]))
    
    return img

print("train set:")
correct = 0
total = 0
for person in train_classes:
    for file in os.listdir("persons-cropped/" + person):
        img = load_image("persons-cropped/" + person + "/" + file)
        adv = create_adv(img, int(inverted_labels[person]))
        preds = model.predict(adv)
        pred = tf.argmax(preds, axis=1)[0].numpy()
        print(person == labels[str(pred)])
        
        if person == labels[str(pred)]:
            correct += 1
        total += 1
print ("correct: " + str(correct) + "(" + str(correct / total) + ")" + " wrong: " + str(total - correct) + "(" + str((total - correct) / total) + ")")
    
print("test set:")
correct = 0
total = 0
for person in test_classes:
    for file in os.listdir("persons-cropped-test/" + person):
        img = load_image("persons-cropped-test/" + person + "/" + file)
        adv = create_adv(img, int(inverted_labels[person]))
        preds = model.predict(adv)
        pred = tf.argmax(preds, axis=1)[0].numpy()
        
        if person == labels[str(pred)]:
            correct += 1
        total += 1
print ("correct: " + str(correct) + "(" + str(correct / total) + ")" + " wrong: " + str(total - correct) + "(" + str((total - correct) / total) + ")")


train set:
True
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
False
True
True
True
True


/usr/lib/python3.8/site-packages/foolbox/attacks/base.py:92: UserWarning: SaliencyMapAttack did not find an adversarial, maybe the model or the criterion is not supported by this attack.
  warnings.warn(


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
False
True
False
False
True
False
True
True
True
False
False
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 